### OpenAI Function Calling

A funcionalidade de chamada de função OpenAI permite definir funções que serão passadas para o LLM. O LLM identificará a função correta para a solicitação e fornecerá parâmetros para a chamada de função.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True) # lê as chaves do arquivo .env e já cria variáveis de ambiente de mesmo nome

True

In [2]:
import openai

def chat(query):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": query}],
    )
    message = response.choices[0].message.content
    return message

In [3]:
query = "Quanto custa um livro de aventuras ?"
message = chat(query)
message

'O preço de um livro de aventuras pode variar bastante dependendo de vários fatores, como o autor, a edição (capa dura ou brochura), a editora e se é um livro novo ou usado. Em geral, você pode encontrar livros de aventuras em lojas físicas ou online com preços que variam de cerca de R$ 20,00 a R$ 100,00 ou mais. Edições especiais ou de colecionador podem custar ainda mais. Para obter um preço específico, é melhor verificar em livrarias locais ou sites de vendas online.'

In [4]:
import json

def get_book_info(book_name: str):
    book_info = {
        "name": book_name,
        "price": "30.00",
    }
    return json.dumps(book_info)

In [5]:
functions = [
    {
        "name": "get_book_info",
        "description": "Obtenha o gênero e o preço de um livro",
        "parameters": {
            "type": "object",
            "properties": {
                "book_name": {
                    "type": "string",
                    "description": "Gênero do livro ",
                },
            },
            "required": ["book_name"],
        },
    }
]

In [6]:
def chat(query):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": query}],
        functions=functions, 
    )
    message = response.choices[0].message
    return message

In [7]:
response = chat("Qual a capital da Itália?")
print(response.content)

A capital da Itália é Roma.


In [8]:
query = "Quanto custa um livro de aventuras?"
message = chat(query)
print(message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"book_name":"aventura"}', name='get_book_info'), tool_calls=None)


In [9]:
if message.function_call:
    book_name = json.loads(message.function_call.arguments).get("book_name")
    print(book_name)
    function_response = get_book_info(
        book_name=book_name
    )
    print(function_response)

aventura
{"name": "aventura", "price": "30.00"}


In [10]:
second_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": query},
        message,
        {
            "role": "function",
            "name": "get_book_info",
            "content": function_response,
        },
    ],
)
second_response

ChatCompletion(id='chatcmpl-BhfSVsnP2rkeafXepz0gr0IvuNpSM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Um livro de aventuras custa aproximadamente R$ 30,00.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1749747171, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=13, prompt_tokens=55, total_tokens=68, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [11]:
second_response.choices[0].message.content

'Um livro de aventuras custa aproximadamente R$ 30,00.'

O mesmo pode ser feito com LangChain

In [12]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

from langchain.prompts.prompt import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain

In [13]:
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm = ChatGroq(model="llama3-8b-8192")

template = """Você é um assistente de chatbot conversando com humanos.

Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["human_input"], template=template)

chain = create_openai_fn_chain(functions, llm, prompt, verbose=False)

D:\Users\abrantes\AppData\Local\Temp\ipykernel_11632\502016012.py:10: LangChainDeprecationWarning: The function `create_openai_fn_chain` was deprecated in LangChain 0.1.1 and will be removed in 1.0. Use :meth:`~create_openai_fn_runnable` instead.
  chain = create_openai_fn_chain(functions, llm, prompt, verbose=False)


In [14]:
message = chain.invoke("Quanto custa um livro de aventura?")
print(message)

{'human_input': 'Quanto custa um livro de aventura?', 'function': {'book_name': 'livro de aventura'}}


In [15]:
print(message["function"]["book_name"])

livro de aventura


In [16]:
book_name = message["function"]["book_name"]
function_response = get_book_info(
        book_name=book_name
)
print(function_response)

{"name": "livro de aventura", "price": "30.00"}


podemos usar Pydantic Classes ao inves de JSON Schemas

In [17]:
from pydantic import BaseModel, Field

class GetBookInfo(BaseModel):
    """Obtenha o gênero e o preço do livro."""

    book_name: str = Field(..., description="O gênero do livro, p.ex. aventura")

pydantic_classes = [GetBookInfo]

In [18]:
chain = create_openai_fn_chain(pydantic_classes, llm, prompt, verbose=False)

In [19]:
chain.invoke("Quanto custa um livro de aventura?")

{'human_input': 'Quanto custa um livro de aventura?',
 'function': GetBookInfo(book_name='aventura')}

Também podemos passar Functions diretamente. Para passar uma função Python diretamente, queremos ter certeza de que nossos parâmetros têm dicas de tipo, temos uma docstring e usamos docstrings no estilo Google Python para descrever os parâmetros.

In [20]:
def get_book_info(book_name: str) -> dict[str, str]:
    """
    Obtenha o gênero e preço de um livro.

    Args:
        book_name: O Gênero do livro, p.ex. aventura.

    Returns:
        Dict[str, str]: A dicionário contendo o gênero book_name e preço do livro.
    """
    book_info = {
        "name": book_name,
        "price": "30.00",
    }
    return book_info

In [21]:
chain = create_openai_fn_chain([get_book_info], llm, prompt, verbose=True)

In [22]:
resposta = chain.run("Quanto custa um livro de aventuras?")
print(resposta)

D:\Users\abrantes\AppData\Local\Temp\ipykernel_11632\4257101381.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = chain.run("Quanto custa um livro de aventuras?")




> Entering new LLMChain chain...
Prompt after formatting:
Você é um assistente de chatbot conversando com humanos.

Human: Quanto custa um livro de aventuras?
AI: 

> Finished chain.
{'book_name': 'livro de aventuras'}


In [23]:
resposta = chain.run("Qual capital da França?")
print(resposta)



> Entering new LLMChain chain...
Prompt after formatting:
Você é um assistente de chatbot conversando com humanos.

Human: Qual capital da França?
AI: 

> Finished chain.
{'book_name': 'Paris'}


In [24]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

database = [
    {"name": "aventura", "price": 32.00},
    {"name": "drama", "price": 40.00},
    {"name": "história", "price": 90.00},
    {"name": "filosofia", "price": 60.00}
]

def get_book_info(book_name: str) -> dict:
    """
    Obtenha o gênero e preço de um livro.

    Args:
        book_name: Gênero de livro

    Returns:
        dict: Um dicionário contendo o gênero e preço do livro e retornando caso gênero não encontrado.
    """
    for book in database:
        if book["name"] == book_name:
            return book
    return {"message": f"Não foi encontrado livro deste gênero {book_name}."}


def add_book(book_name: str, price: float) -> dict:
    """
    Adicione um novo gênero book_name ao banco de dados.

    Args:
        book_name: Novo Gênero de livro a ser adicionado.
        price: preço do novo gênero de livro adicionado.

    Returns:
        dict: uma mensagems que informa resultado da adição.
    """
    for book in database:
        if book["name"] == book_name:
            return {"message": f"Gênero de Livro {book_name} já existe no banco de dados."}

    database.append({"name": book_name, "price": price})
    
    return {"message": f"Gênero de livro {book_name} adicionado com sucesso!"}


In [25]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

template = """Você é um assistente tendo uma conversa com humanos

Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["human_input"], template=template)

chain = create_openai_fn_chain([get_book_info,add_book], llm, prompt, verbose=False)


In [26]:
resposta = chain.run("Quanto custa um livro de aventuras?")
print(resposta)

{'arguments': {'book_name': 'aventuras'}, 'name': 'get_book_info'}


In [27]:
resposta = chain.run("Eu quero adicionar gênero de livro de artes por 100.80 ")
print(resposta)
resposta_save = resposta

{'arguments': {'book_name': 'artes', 'price': 100.8}, 'name': 'add_book'}


In [28]:
try:
    result2 = chain.run("Qual a capital da Itália?")
    print(result2)
except Exception as e:
    print(e)

Could not parse function call: 'function_call'
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [29]:
resposta = chain.run("Quanto custa um livro de artes?")
print(resposta)

{'arguments': {'book_name': 'artes'}, 'name': 'get_book_info'}


In [30]:
print(get_book_info('história'))

{'name': 'história', 'price': 90.0}


In [31]:
print(get_book_info('artes'))

{'message': 'Não foi encontrado livro deste gênero artes.'}


In [32]:
print(resposta_save)
if resposta_save["name"]=='add_book':
    book_name = resposta_save["arguments"]["book_name"]
    price = resposta_save["arguments"]["price"]
    resultado = add_book(book_name, price) 
    print(resultado)


{'arguments': {'book_name': 'artes', 'price': 100.8}, 'name': 'add_book'}
{'message': 'Gênero de livro artes adicionado com sucesso!'}


In [33]:
print(get_book_info('artes'))

{'name': 'artes', 'price': 100.8}


In [34]:
database

[{'name': 'aventura', 'price': 32.0},
 {'name': 'drama', 'price': 40.0},
 {'name': 'história', 'price': 90.0},
 {'name': 'filosofia', 'price': 60.0},
 {'name': 'artes', 'price': 100.8}]